# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps.datasets
import os
from api_keys import gkey




In [12]:
# Import gkey key
gmaps.configure(gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
city_list_file = "cityDataFile.csv"
city_list_file_df = pd.read_csv(city_list_file)
city_list_file_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bocaiuva,-17.11,-43.81,82.40,24,0,12.75,BR,1599946895
1,1,penzance,50.12,-5.54,59.00,100,86,6.93,GB,1599946593
2,2,manyana,-23.40,21.72,59.90,31,0,8.37,BW,1599946895
3,3,torbay,47.67,-52.73,57.00,54,20,16.11,CA,1599946524
4,4,hithadhoo,-0.60,73.08,81.68,73,25,7.29,MV,1599946519
...,...,...,...,...,...,...,...,...,...,...
561,561,nanortalik,60.14,-45.24,38.82,73,2,3.18,GL,1599946582
562,562,bonthe,7.53,-12.51,76.24,92,95,5.55,SL,1599946518
563,563,jamestown,42.10,-79.24,75.00,73,90,9.17,US,1599946460
564,564,luena,-11.78,19.92,63.46,30,29,5.21,AO,1599947007


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
locations = city_list_file_df[["Lat", "Lng"]]
humidity = city_list_file_df["Humidity"].astype(float)

In [15]:
fig = gmaps.figure()

# Create heat layer
humidity_heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(humidity_heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
maxlow_locations_df = city_list_file_df[city_list_file_df['Max Temp'] > 70] 
vacation_temps_df = maxlow_locations_df[maxlow_locations_df['Max Temp'] <85]
vacation_temps_df.dropna(how="any")
vacation_temps_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bocaiuva,-17.11,-43.81,82.40,24,0,12.75,BR,1599946895
4,4,hithadhoo,-0.60,73.08,81.68,73,25,7.29,MV,1599946519
7,7,los llanos de aridane,28.66,-17.92,75.20,78,90,2.24,ES,1599946637
8,8,mogadishu,2.04,45.34,78.03,83,100,19.75,SO,1599946889
11,11,omboue,-1.57,9.26,75.74,79,99,8.93,GA,1599946588
...,...,...,...,...,...,...,...,...,...,...
555,555,wanlaweyn,2.62,44.89,77.45,76,100,17.60,SO,1599946986
557,557,los llanos de aridane,28.66,-17.92,75.20,78,90,2.24,ES,1599946637
558,558,dakar,14.69,-17.44,78.80,88,20,3.36,SN,1599946968
562,562,bonthe,7.53,-12.51,76.24,92,95,5.55,SL,1599946518


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
vacation_temps_df["Hotel Name"]= ""
vacation_temps_df

C:\Users\Maher\anaconda3\envs\bcs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,bocaiuva,-17.11,-43.81,82.40,24,0,12.75,BR,1599946895,
4,4,hithadhoo,-0.60,73.08,81.68,73,25,7.29,MV,1599946519,
7,7,los llanos de aridane,28.66,-17.92,75.20,78,90,2.24,ES,1599946637,
8,8,mogadishu,2.04,45.34,78.03,83,100,19.75,SO,1599946889,
11,11,omboue,-1.57,9.26,75.74,79,99,8.93,GA,1599946588,
...,...,...,...,...,...,...,...,...,...,...,...
555,555,wanlaweyn,2.62,44.89,77.45,76,100,17.60,SO,1599946986,
557,557,los llanos de aridane,28.66,-17.92,75.20,78,90,2.24,ES,1599946637,
558,558,dakar,14.69,-17.44,78.80,88,20,3.36,SN,1599946968,
562,562,bonthe,7.53,-12.51,76.24,92,95,5.55,SL,1599946518,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
